# **Record TBRC Video and Data**
Author: Michael Kainola

Date: August 1, 2022

This notebook includes the code used to record the video footage of the TBRC, quantify the image's luminosity, and save the data to csv. 

Notes:
* This code references a custom external python module (cv_support.py) which is shared between experiment \#1 and \#2.
* This code will fail (gracefully) unless the required PICAM is detected. If this occurs, any generated video files will not work.

<br>

**Step 1. Load the external dependencies.**

In [16]:
# import standard modules
from datetime import datetime #used for timestamping
import cv2 #used for capturing video
import os #used for creating directories
import sys #used for importing custom module

# import custom module
sys.path.insert(0, '../Common Files') #add Common Files folder to the system path
from cv_support import * #custom function for reading/writing CSVs, recording video

<br>

**Step 2. Define application parameters.**

In [17]:
# create folder string; used later when storing the video recording
prefix = "../Common Files/Videos/PICAM "
folder_now = datetime.now().strftime("%Y%m%d %H%M%S")
folder_name = prefix + str(folder_now)

#ouput file prefix
filename = 'PICAM '

# specify codec to be used when writing video
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# import the ROIs for which the luminosity will be recorded
rois = readROICoords("./Inputs/roi_coords.csv")

# extract the luminosity values from the returned object
_roi1 = rois[0]
_roi2 = rois[1]
_roi3 = rois[2]

# configure video length and number of videos to output
vid_length_seconds = 120 #length of each video, in seconds
video_count = 1 #number of videos to record

frame_count = vid_length_seconds * 10 #total frames to record

Reading ROI coordinates from: ./Inputs/roi_coords.csv



<br>

**Step 3. Define the function to record the video, extract the luminosity and save the csv.**

In [18]:
# function containing core application flow, called on application initialization
def capture_video():
    
    #for storing luminosity data
    avg_image = []
    avg_roi1 = []
    avg_roi2 = []
    avg_roi3 = []
    
    # create capture object using helper functions
    cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)
    
    # create output object, specify the codec, file format, and video dimensions
    now = datetime.now().strftime("%Y%m%d %H%M%S")
    out = cv2.VideoWriter(folder_name + "/" + filename + " - " + str(now) + ".avi",fourcc, 10,(640,480),True)
    
    # create csv data file, for storing quantified luminosity values
    createVideoDataFile(folder_name + "/LUMINOSITY - " + str(now) + ".csv")
        
    # if camera available, start capturing video
    if cap.isOpened():
        
        x = 0
        
        # capture frames until frame_count is met
        while x < frame_count:
            
            # capture the frame
            ret_val, img = cap.read()

            # convert image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
            # specify the ROI (Region of interest) coordinates
            roi1 = gray[_roi1.top:_roi1.top + _roi1.height, _roi1.left:_roi1.left + _roi1.width]
            roi2 = gray[_roi2.top:_roi2.top + _roi2.height, _roi2.left:_roi2.left + _roi2.width]
            roi3 = gray[_roi3.top:_roi3.top + _roi3.height, _roi3.left:_roi3.left + _roi3.width]
            
            # construct the row to be written to csv
            row = [
                [
                    x + 1, #frame
                    cv2.mean(roi1)[0], #roi1, average pixel value
                    cv2.mean(roi2)[0], #roi2, average pixel value
                    cv2.mean(roi3)[0], #roi3, average pixel value
                    cv2.mean(gray)[0], #image, average pixel value
                    datetime.now() #timestamp
                ]
            ]
            
            #write the data to csv
            writeVideoData(row)

            #record the frame
            out.write(img)
            
            x = x + 1

        # clean up objects otherwise camera becomes inaccessible
        cap.release()
        out.release()
    
    # if camera not available, notify the user
    else:
        print("Unable to open camera")
    readVideoData()

<br>

**Step 4. Run the main application routine.**

In [19]:
#main application routine
if __name__ == "__main__":
    
    #create new folder to store the video and csv (luminosity) data
    os.mkdir(folder_name)
        
    #loop until video_count is met
    for y in range(video_count):
        
        #call function to capture and save video
        capture_video()
        
        #provide user with feedback after each video file has been created
        print("\n")
        print("*********************")
        print("*********************")
        print("Video " + str(y + 1) + "/" + str(video_count) + " complete.")
        print("*********************")
        print("*********************")
        print("\n")

Unable to open camera
Reading file: ../Common Files/Videos/PICAM 20220801 151325/LUMINOSITY - 20220801 151338.csv

['FRAME', 'ROI1', 'ROI2', 'ROI3', 'IMAGE', 'TIMESTAMP']


*********************
*********************
Video 1/1 complete.
*********************
*********************


